In [2]:
import pandas as pd
import numpy as np
import os

Aqui iremos extrair features das imagens em raio-x de pessoas saudáveis

In [2]:
img_list=os.listdir('C:/Users/jocaj/Desktop/chest_xray/train/NORMAL/')
img_dir='C:/Users/jocaj/Desktop/chest_xray/train/NORMAL/'

In [3]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input as preprocess_vgg16_input

In [4]:
def process_image2arr(path, img_dims=(224, 224)):
    img = image.load_img(path, target_size=img_dims)
    img_arr = image.img_to_array(img)
    img_arr = np.expand_dims(img_arr, axis=0)
    img_arr = preprocess_vgg16_input(img_arr)
    return img_arr

In [5]:
from keras.applications import vgg16
from keras.models import Model


vgg_model = vgg16.VGG16(include_top=True, weights='imagenet', 
                        input_shape=(224, 224, 3))
vgg_model.layers.pop()
output = vgg_model.layers[-1].output
vgg_model = Model(vgg_model.input, output)
vgg_model.trainable = False

In [6]:
def extract_tl_features_vgg(model, image_file_name, image_dir=img_dir):
    
    pr_img = process_image2arr(image_dir+image_file_name)
    tl_features = model.predict(pr_img)
    tl_features = np.reshape(tl_features, tl_features.shape[1])
    return tl_features

In [10]:
img_features = dict()
img_names = []

In [11]:
#montar a base de dados com os features das imagens (demora)
for img in img_list:
    img_features[img] = extract_tl_features_vgg(model=vgg_model, image_file_name=img, image_dir=img_dir)

In [13]:
dataset=pd.DataFrame.from_dict(img_features, orient='index')

In [17]:
#Colocar uma variável para ser categórica no modelo
dataset['Tipo']='Normal' 
dataset.head()

,0,1,2,3,4,5,6,7,8,9,...,991,992,993,994,995,996,997,998,999,Tipo
IM-0115-0001.jpeg,8.252476e-07,0.000005,0.000015,0.000020,0.000080,0.000004,0.000011,1.685664e-06,5.402282e-07,4.608167e-08,...,0.000007,0.000003,4.957352e-07,0.000003,2.758236e-06,0.000022,0.000001,0.000151,0.013038,Normal
IM-0117-0001.jpeg,2.291649e-06,0.000060,0.000020,0.000031,0.000010,0.000004,0.000002,3.265323e-06,1.816521e-06,1.457775e-07,...,0.000001,0.000003,3.029277e-07,0.000010,1.707152e-06,0.000118,0.000001,0.000904,0.000370,Normal
IM-0119-0001.jpeg,3.647016e-06,0.000012,0.000014,0.000053,0.000023,0.000007,0.000008,8.847090e-07,3.783455e-07,2.799540e-08,...,0.000004,0.000004,7.258530e-07,0.000010,2.211052e-06,0.000076,0.000003,0.000400,0.000782,Normal
IM-0122-0001.jpeg,2.127450e-06,0.000008,0.000012,0.000113,0.000014,0.000003,0.000009,1.845175e-07,5.583694e-08,2.580254e-09,...,0.000001,0.000006,8.992288e-08,0.000003,4.752495e-07,0.000071,0.000003,0.000788,0.001977,Normal
IM-0125-0001.jpeg,1.010185e-06,0.000021,0.000054,0.000315,0.000203,0.000008,0.000004,4.416500e-06,9.258724e-07,3.217959e-07,...,0.000005,0.000003,9.132696e-07,0.000012,1.822083e-06,0.000125,0.000002,0.000527,0.001200,Normal


Repete-se o processo para imagens de raio-x de pessoas com pneumonia

In [18]:
img_list=os.listdir('C:/Users/jocaj/Desktop/chest_xray/train/PNEUMONIA/')
img_dir='C:/Users/jocaj/Desktop/chest_xray/train/PNEUMONIA/'

In [19]:
img_features = dict()
img_names = []

In [20]:
#montar a base de dados com os features das imagens (demora)
for img in img_list:
    img_features[img] = extract_tl_features_vgg(model=vgg_model, image_file_name=img, image_dir=img_dir)

In [21]:
pneumo=pd.DataFrame.from_dict(img_features, orient='index')

In [23]:
pneumo['Tipo']='Pneumonia'
pneumo.head()

,0,1,2,3,4,5,6,7,8,9,...,991,992,993,994,995,996,997,998,999,Tipo
person1000_bacteria_2931.jpeg,6.147668e-05,0.000018,0.006439,0.012946,0.000984,0.000315,0.000825,0.000003,1.782101e-06,5.799765e-07,...,0.000004,0.000010,7.377552e-07,0.000004,0.000003,0.000072,0.000007,0.000507,0.001085,Pneumonia
person1000_virus_1681.jpeg,4.783651e-06,0.000027,0.001903,0.004457,0.000340,0.000021,0.000035,0.000003,2.118637e-06,4.623182e-07,...,0.000002,0.000008,3.489384e-07,0.000005,0.000002,0.000071,0.000004,0.000258,0.015702,Pneumonia
person1001_bacteria_2932.jpeg,2.904727e-06,0.000002,0.000131,0.000898,0.000118,0.000034,0.000007,0.000002,1.148636e-06,1.863359e-07,...,0.000006,0.000001,1.860307e-06,0.000096,0.000002,0.000220,0.000003,0.000293,0.000199,Pneumonia
person1002_bacteria_2933.jpeg,6.935853e-07,0.000019,0.000028,0.000088,0.000145,0.000007,0.000003,0.000003,4.013601e-07,7.507899e-08,...,0.000004,0.000002,6.002315e-07,0.000003,0.000002,0.001086,0.000002,0.000318,0.000226,Pneumonia
person1003_bacteria_2934.jpeg,3.696640e-06,0.000078,0.000341,0.000223,0.000879,0.000035,0.000056,0.000015,6.528738e-06,2.884869e-06,...,0.000010,0.000017,6.004425e-07,0.000006,0.000005,0.000219,0.000007,0.000768,0.001073,Pneumonia


In [3]:
pd.concat(dataset,pneumo)

NameError: name 'dataset' is not defined